In [1]:
from PIL import Image
import os
import torch
import matplotlib.pyplot as plt

In [2]:
from patched_clip.patched_clip import CLIP_args, get_clip_embeddings, load_clip, save_clip_embeddings

In [3]:
image_paths = ['test.jpg']
device = 'cpu'

In [4]:
# CLIP_args.clip_model_name = CLIPArgs.model_type
# CLIP_args.device = device
high_res_feature = True

# Load images into pillow images
images = [Image.open(image_path) for image_path in image_paths]

# Get CLIP embeddings
with torch.no_grad():
    descriptors = get_clip_embeddings(images, to_cpu=False, skip_center_crop=high_res_feature)
    descriptors = descriptors.to(device)

embeddings = descriptors.numpy()

ML_LOGGER_USER is not set. This is required for online usage.
timing <load_clip_model>: 4.571E+00s
timing <clip_preprocess_images>: 1.946E-01s
timing <get_clip_embeddings>: 1.096E+00s
clip_process_image_mean_time: 1.2925954579986865


In [4]:
save_clip_embeddings(image_paths)

ML_LOGGER_USER is not set. This is required for online usage.
══════════════════════════════════════════
     CLIP_args      
────────────────────┬─────────────────────
  clip_model_name   │ ViT-L/14@336px      
visualize_embeddings│ True                
  visualize_every   │ 5                   
     batch_size     │ 64                  
       device       │ cpu                 
    feature_dir     │ feat                
   visualize_dir    │ clip_viz            
     patch_size     │ 14                  
════════════════════╧═════════════════════
Loaded 1 images from list
Image resolution is (4032, 3024)
timing <load_clip_model>: 4.517E+00s
timing <clip_preprocess_images>: 1.938E-01s
timing <get_clip_embeddings>: 7.632E-01s
clip_process_image_mean_time: 0.958800541004166
embedding shape = (1, 768, 24, 24)
timing <write_clip_embeddings>: 1.420E-02s
Wrote CLIP embeddings to ./feat/clip.npy
Saved image at figures/test_viz.png.
timing <visualize_clip_embeddings>: 4.846E+00s
Wrote CLIP p

'./feat/clip.npy'

In [5]:
descriptors.shape

torch.Size([1, 768, 24, 32])

In [6]:
from patched_clip.utils import visualize_embedding_pca

In [16]:
model, preprocess = load_clip()

In [17]:
preprocess

Compose(
    Resize(size=336, interpolation=bicubic, max_size=None, antialias=warn)
    CenterCrop(size=(336, 336))
    <function _convert_image_to_rgb at 0x15453ec00>
    ToTensor()
    Normalize(mean=(0.48145466, 0.4578275, 0.40821073), std=(0.26862954, 0.26130258, 0.27577711))
)

In [14]:
visualize_embedding_pca(
                image_paths,
                images,
                embeddings,
                preprocess=load_clip()[1],
                visualize_every=CLIP_args.visualize_every,
                patch_size=14,
            )

ValueError: cannot reshape array of size 2304 into shape (24,24,3)